In [1]:
include("functions.jl")

seq_des (generic function with 1 method)

In [2]:
magic = readdlm("magic04.data", ',', Any, '\n')
loc = findall(x->x=="g",magic[:,11])
magic[loc,11] .= 1
loc = findall(x->x=="h",magic[:,11])
magic[loc,11] .= 0
magic = convert(Array{Float64}, magic)

#=
X = magic
y = X[:,11]
X = X[:,1:10]
n, p = size(X)
=#

#class = magic[:,11]
#magic[:,11] .= 1
#n, p = size(magic)

class = magic[:,11]
magic = magic[:,1:10]
n, p = size(magic)

(19020, 10)

magic = randn(10^4, 10)
class = round.(rand(10^4))
n, p = size(magic)

In [3]:
n0 = 100
h = 200
nq = 1 #30
ϵ = 10^-2
α= 0.5

0.5

## Greedy algorithm

In [4]:
Random.seed!(1)
Ξs = sample(1:n,n0, replace = false)
Ξv = p
crit = Inf

while crit >= ϵ && length(Ξv) <= p
    t1 = time()
    β̂ = irls(magic[Ξs, Ξv], class[Ξs], 10^-6)
    for t = 1:nq
        p̂ = μ(magic[setdiff(1:n,Ξs), Ξv], β̂)
        d = abs.(p̂ .- α)
        Ξi = setdiff(1:n,Ξs)[sortperm(d)[1:h]]
        
        select = 0
        max_val = -Inf
        
        p̂ = μ(magic[Ξs, Ξv], β̂)
        w = p̂ .* (1 .- p̂)
        M = magic[Ξs, Ξv]'*Diagonal(w)*magic[Ξs, Ξv]
        for i = Ξi
            w = μ(magic[i:i, Ξv], β̂)[1]
            w = w * (1 - w)
            val = logdet(M + w*magic[i, Ξv]*magic[i, Ξv]')
            if val > max_val
                select = i
                max_val = val
            end
        end
        Ξs = [Ξs; select]
        β̂ = irls(magic[Ξs, Ξv], class[Ξs], 10^-6)
    end

    p̂ = μ(magic[Ξs, Ξv], β̂)
    w = p̂ .* (1 .- p̂)
    M0 = (det(1/length(Ξs) *magic[Ξs, Ξv]'*Diagonal(w)*magic[Ξs, Ξv]))^(1/length(Ξv))
    if length(Ξv) < p 
        g = abs.(magic[Ξs, setdiff(1:p,Ξv)]'*(class[Ξs] - p̂))
        u = findmax(g)[2]
        u = setdiff(1:p,Ξv)[u]
        Ξv = [Ξv; u]
        β̂ = irls(magic[Ξs, Ξv], class[Ξs], 10^-6)
        p̂ = μ(magic[Ξs, Ξv], β̂)
        w = p̂ .* (1 .- p̂)
    end
    M1 = (det(1/length(Ξs) *magic[Ξs, Ξv]'*Diagonal(w)*magic[Ξs, Ξv]))^(1/length(Ξv))
    crit = abs(M1 - M0) / M0
    if crit < ϵ && length(Ξv) < p
        Ξv = setdiff(Ξv, u)
    end
    println(time()-t1)
end
s1 = Ξs
v1 = Ξv

4.677770137786865
0.45446014404296875
0.005925893783569336
0.013895988464355469
0.0060198307037353516
0.006773948669433594
0.007117033004760742
0.007246971130371094
0.007451057434082031
0.010803937911987305


10-element Array{Int64,1}:
 10
  6
  1
  9
  8
  2
  7
  3
  4
  5

## SOCP algorithm

In [5]:
Random.seed!(1)
Ξs = sample(1:n,n0, replace = false)
Ξv = p
crit = Inf

while crit >= ϵ && length(Ξv) <= p
    t1 = time()
    β̂ = irls(magic[Ξs, Ξv], class[Ξs], 10^-6)
    p̂ = μ(magic[setdiff(1:n,Ξs), Ξv], β̂)
    d = abs.(p̂ .- α)
    Ξi = setdiff(1:n,Ξs)[sortperm(d)[1:h]]

    p̂ = μ(magic[[Ξi; Ξs], Ξv], β̂)
    w = p̂ .* (1 .- p̂)
    select = sagnol_D(Diagonal(sqrt.(w[1:h]))*magic[Ξi, Ξv], Diagonal(sqrt.(w[h+1:end]))*magic[Ξs, Ξv], nq, IC = 1)
    select = Ξi[findall(x->x>0.5, select)]
    Ξs = [Ξs; select]
    
    β̂ = irls(magic[Ξs, Ξv], class[Ξs], 10^-6)
    p̂ = μ(magic[Ξs, Ξv], β̂)
    w = p̂ .* (1 .- p̂)
    M0 = (det(1/length(Ξs) *magic[Ξs, Ξv]'*Diagonal(w)*magic[Ξs, Ξv]))^(1/length(Ξv))
    if length(Ξv) < p 
        g = abs.(magic[Ξs, setdiff(1:p,Ξv)]'*(class[Ξs] - p̂))
        u = findmax(g)[2]
        u = setdiff(1:p,Ξv)[u]
        Ξv = [Ξv; u]
        β̂ = irls(magic[Ξs, Ξv], class[Ξs], 10^-6)
        p̂ = μ(magic[Ξs, Ξv], β̂)
        w = p̂ .* (1 .- p̂)
    end
    M1 = (det(1/length(Ξs) *magic[Ξs, Ξv]'*Diagonal(w)*magic[Ξs, Ξv]))^(1/length(Ξv))
    crit = abs(M1 - M0) / M0
    if crit < ϵ && length(Ξv) < p
        Ξv = setdiff(Ξv, u)
    end
    println(time()-t1)
end
s2 = Ξs
v2 = Ξv

17.46479105949402
5.001065969467163
1.688715934753418
4.812160968780518
6.888355016708374
3.719360113143921
11.80534315109253
6.664076089859009
9.422388076782227
20.74282693862915


10-element Array{Int64,1}:
 10
  6
  9
  1
  7
  8
  2
  3
  4
  5

In [6]:
println(s1[101:end])
println(s2[101:end])

[16769, 12507, 14895, 14090, 17171, 17203, 16047, 13902, 14066, 18104]
[16769, 15749, 18714, 7391, 16495, 18809, 15111, 17867, 4735, 12534]


In [7]:
println(v1)
println(v2)

[10, 6, 1, 9, 8, 2, 7, 3, 4, 5]
[10, 6, 9, 1, 7, 8, 2, 3, 4, 5]


In [8]:
addon = 1
Ξs = s1[1:100+addon]
Ξv = v1[1:1+addon]
crit = Inf

β̂ = irls(magic[Ξs, Ξv], class[Ξs], 10^-6)
p̂ = μ(magic[setdiff(1:n,Ξs), Ξv], β̂)
d = abs.(p̂ .- α)
Ξi = setdiff(1:n,Ξs)[sortperm(d)[1:h]]

p̂ = μ(magic[[Ξi; Ξs], Ξv], β̂)
w = p̂ .* (1 .- p̂)
mat = Diagonal(sqrt.(w[1:h]))*magic[Ξi, Ξv]
mat0 = Diagonal(sqrt.(w[h+1:end]))*magic[Ξs, Ξv]
nq

1

In [9]:
select = sagnol_D(mat, mat0, nq, IC = 1, verbose = 0)
Ξi[findall(x->x>0.5, select)]

1-element Array{Int64,1}:
 15749

In [10]:
select = sagnol_D(mat, mat0, nq, IC = 1, verbose = 0)
Ξi[findall(x->x>0.5, select)]

1-element Array{Int64,1}:
 15749